In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from keras import backend as K

import time

/home/danielzhang/Projects/alzheimers_sinai/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def encoder(input_encoder):
	inputs = keras.Input(shape=input_encoder, name='input_layer')

	# Block-1
	x = layers.Dense(100, name='conv_1')(inputs)
	x = layers.LeakyReLU(name='lrelu_1')(x)

	# Block-2
	x = layers.Dense(75, name='conv_2')(x)
	x = layers.LeakyReLU(name='lrelu_2')(x)

	# Block-3
	x = layers.Dense(75, name='conv_3')(x)
	x = layers.LeakyReLU(name='lrelu_3')(x)


	# Block-4
	x = layers.Dense(50, name='conv_4')(x)
	x = layers.LeakyReLU(name='lrelu_4')(x)

	# Final Block
	flatten = x
	mean = layers.Dense(2, name='mean')(flatten)
	log_var = layers.Dense(2, name='log_var')(flatten)
	model = tf.keras.Model(inputs, (mean, log_var), name="Encoder")

	return model


def sampling(input_1,input_2):
	mean = keras.Input(shape=input_1, name='input_layer1')
	log_var = keras.Input(shape=input_2, name='input_layer2')
	out = layers.Lambda(sampling_reparameterization_model, name='encoder_output')([mean, log_var])
	enc_2 = tf.keras.Model([mean,log_var], out,  name="Encoder_2")

	return enc_2


def sampling_reparameterization_model(distribution_params):
    mean, log_var = distribution_params
    epsilon = K.random_normal(shape=K.shape(mean), mean=0., stddev=1.)
    z = mean + K.exp(log_var / 2) * epsilon

    return z


# replace the decoder with the supervised learning method
def decoder(input_decoder):
		
	inputs = keras.Input(shape=input_decoder, name='input_layer')
	x = layers.Dense(50, name='dense_1')(inputs)
	
	# Block-1
	x = layers.Dense(75,name='conv_transpose_1')(x)
	x = layers.BatchNormalization(name='bn_1')(x)
	x = layers.LeakyReLU(name='lrelu_1')(x)
	
	# Block-2
	x = layers.Dense(75, name='conv_transpose_2')(x)
	x = layers.BatchNormalization(name='bn_2')(x)
	x = layers.LeakyReLU(name='lrelu_2')(x)		

	# INSERT NORMAL SUPERVISED LEARNING HERE?
	# Block-4
	outputs = layers.Dense(100, name='conv_transpose_4')(x)
	model = tf.keras.Model(inputs, outputs, name="Decoder")
	return model	

optimizer = tf.keras.optimizers.Adam(lr = 0.0005)

def mse_loss(y_true, y_pred):
	r_loss = K.mean(K.square(y_true - y_pred), axis = [1,2,3])
	return 1000 * r_loss
	

def kl_loss(mean, log_var):
	kl_loss =  -0.5 * K.sum(1 + log_var - K.square(mean) - K.exp(log_var), axis = 1)
	return kl_loss
	

def vae_loss(y_true, y_pred, mean, log_var):
	r_loss = mse_loss(y_true, y_pred)
	kl_loss = kl_loss(mean, log_var)
	return  r_loss + kl_loss

@tf.function
def train_step(images):
	
	with tf.GradientTape() as enc, tf.GradientTape() as dec:
		
		mean, log_var = enc(images, training=True)
		latent = sampling([mean, log_var])
		generated_images = dec(latent, training=True)
		loss = vae_loss(images, generated_images, mean, log_var)
	
			
	gradients_of_enc = encoder.gradient(loss, enc.trainable_variables)
	gradients_of_dec = decoder.gradient(loss, dec.trainable_variables)
		
		
	optimizer.apply_gradients(zip(gradients_of_enc, enc.trainable_variables))
	optimizer.apply_gradients(zip(gradients_of_dec, dec.trainable_variables))
	return loss


def train(dataset, epochs):
	for epoch in range(epochs):
		start = time.time()
		for image_batch in dataset:
			train_step(image_batch)
	
	print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))



In [2]:
input_size = 256
n_classes = 3

lr_l1 = LogisticRegression(penalty="l1")
svm = SVC()
rf = RandomForestClassifier(n_estimators=500)

# 1400, 300
# 1130, 300
# 1260, 190
# 1000, 280
# 185, 50 (n)
# 1111, 635
# 1008, 454
# 1005, 448 (nw)
# 180, 566 (n f)
# 190, 400 (n t)
# 180 630 (n s)
dnn = keras.Sequential([
    layers.Dense(input_size),
    layers.Dense(100),
    layers.Dense(75),
    layers.Dense(75),
    layers.Dense(50),
    layers.Dense(n_classes)
])
dnn.compile(optimizer='adam', 
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=[keras.metrics.CategoricalAccuracy()])





2022-07-18 15:44:49.642431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-18 15:44:49.642699: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-18 15:44:49.642747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-18 15:44:49.642784: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-18 15:44:49.642824: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [12]:
mnist = tfds.image_classification.MNISTCorrupted
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(60000, 28**2)
x_test = x_test.reshape(10000, 28**2)


AttributeError: module 'keras.api._v2.keras.datasets' has no attribute 'mnist_corrupted'

In [9]:
x_train.shape

(60000, 784)